# Advanced techniques to improve RAG 


* video tutorial: https://www.youtube.com/playlist?list=PLfaIDFEXuae2LXbO1_PKyVJiQ23ZztA0x
* repository: https://github.com/langchain-ai/rag-from-scratch

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
# Establecemos los modelos que vamos a usar
EMBD_MODEL="text-embedding-3-small"
LLM_MODEL="gpt-4o-mini"

#### Load the necessary modules

In [27]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

#### **Indexing**

##### 1. Load from web

In [28]:
# Load document, in this case from web page
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
docs = loader.load()

##### 2. Split the document in small chunks of text

In [29]:
# Split document in small chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

##### Convert the chunks of text in numbers (embeddings) and load them into a vector database

In [30]:
# Convert small chunks in numbers (embeddings) and store in vector database
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings(model=EMBD_MODEL),
)

#### **Retrieve and Generate**

In [31]:
# Set the retriever
retriever = vectorstore.as_retriever()

In [32]:
# LLM
llm = ChatOpenAI(model_name=LLM_MODEL, temperature=0)

In [33]:
# Retrieve
question = "What is task decomposition for LLM agents?"

### Advanced techniques to improve RAG Performance

#### 1. Query translation 
* Improving a question before it might lead to a low quality anwser from our RAG application

#### Alternative ways to transform the initial question:
* Convert the question in multiple similar questions
    * Multi-Query
    * RAG Fusion
* Convert the question in (progressive or not) subquestions
* Step-back questions (step-back prompting): go one step back and make the questions that precede or originate the user's question. Example:
    * User question: where were Mozart born?
    * Step-back question: who was Mozart? 

## Query translation technique #1: Multi-Query
* Convert the question into several similar questions
* Get the RAG answer for each of them
* Analyze all the answers and produce a final one 

#### Create a prompt template to create 5 similar questions to the question provided by the user

In [9]:
# Multi Query: Different Perspectives
from langchain.prompts import ChatPromptTemplate


template = """You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: {question}"""

prompt_perspectives = ChatPromptTemplate.from_template(template)

#### Define a chain to generate the 5 similar questions

In [34]:
generate_queries = (
    prompt_perspectives 
    | llm
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)
queries = generate_queries.invoke(question)
queries

['What does task decomposition mean in the context of LLM agents?  ',
 'How do LLM agents utilize task decomposition in their processes?  ',
 'Can you explain the concept of task decomposition as it relates to LLM agents?  ',
 'In what ways does task decomposition apply to agents powered by LLMs?  ',
 'What are the benefits of task decomposition for agents using large language models?  ']

#### Get the RAG answer for each of them

In [41]:
from langchain.load import dumps, loads

def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]


retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question":question})
len(docs)

5

#### Analyze all the answers and produce a final one

In [16]:
from operator import itemgetter

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

"Task decomposition for LLM agents involves breaking down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks. This process allows the agent to transform big tasks into multiple manageable tasks, shedding light on the interpretation of the model's thinking process."

## Query translation technique #2: RAG Fusion
* Same as Multi-Query, but ranking the resulting questions.

#### Same as before: create a prompt template to create 4 similar questions to the question provided by the user

In [37]:
# RAG-Fusion: Related
template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output (4 queries):"""
prompt_rag_fusion = ChatPromptTemplate.from_template(template)

#### Same as before: define a chain to generate the 4 similar questions

In [38]:
generate_queries = (
    prompt_rag_fusion 
    | llm
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

In [19]:
from langchain.load import dumps, loads

# https://medium.com/@nageshmashette32/langchain-rag-fusion-advance-rag-32eefc63da99

def reciprocal_rank_fusion(results: list[list], k=60):
    """ Reciprocal_rank_fusion that takes multiple lists of ranked documents 
        and an optional parameter k used in the RRF formula """
    
    # Initialize a dictionary to hold fused scores for each unique document
    fused_scores = {}

    # Iterate through each list of ranked documents
    for docs in results:
        # Iterate through each document in the list, with its rank (position in the list)
        for rank, doc in enumerate(docs):
            # Convert the document to a string format to use as a key (assumes documents can be serialized to JSON)
            doc_str = dumps(doc)
            # If the document is not yet in the fused_scores dictionary, add it with an initial score of 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Retrieve the current score of the document, if any
            previous_score = fused_scores[doc_str]
            # Update the score of the document using the RRF formula: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # Sort the documents based on their fused scores in descending order to get the final reranked results
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # Return the reranked results as a list of tuples, each containing the document and its fused score
    return reranked_results

retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion
docs = retrieval_chain_rag_fusion.invoke({"question": question})
len(docs)

7

In [42]:
from operator import itemgetter

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain_rag_fusion, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

'Task decomposition for LLM (large language model) agents involves breaking down complex tasks into smaller, more manageable steps. This process is enhanced by techniques such as Chain of Thought (CoT), where the model is prompted to "think step by step," allowing it to utilize more computational resources to simplify difficult tasks. By transforming large tasks into multiple smaller tasks, task decomposition not only makes the overall task easier to handle but also provides insight into the model\'s reasoning process. Additionally, task decomposition can be achieved through various methods, including simple prompting, task-specific instructions, or human inputs.'

## Query translation technique #3: Decomposition
* Decompose de question into progressive sub-questions.
https://python.langchain.com/v0.1/docs/use_cases/query_analysis/techniques/decomposition/

#### Create a prompt template to create 3 subquestions from the question provided by the user

In [54]:
# Decomposition
template = """You are a helpful assistant that generates multiple sub-questions related to an input question. \n
The goal is to break down the input into a set of sub-problems / sub-questions that can be answers in isolation. \n

Perform query decomposition. Given a user question, break it down into distinct sub questions that \
you need to answer in order to answer the original question.

Generate multiple search sub-queries related to: {question} \n
Output (3 queries):"""
prompt_decomposition = ChatPromptTemplate.from_template(template)

In [64]:

# Chain
generate_queries_decomposition = ( prompt_decomposition | llm | StrOutputParser() | (lambda x: x.split("\n")))

# Run
question = "How to Task Decomposition and add Memory"

# These are the subquestions:
questions = generate_queries_decomposition.invoke({"question":question})

In [65]:
#print the subquestions:
questions

['1. What are the different methods of task decomposition?',
 '2. How does task decomposition help in improving memory retention?',
 '3. Can you provide examples of task decomposition techniques that aid memory enhancement?']

#### Option 1: answer each of the subquestions using the response to the previous subquestion as context.

In [66]:
# Prompt
template = """Here is the question you need to answer:

\n --- \n {question} \n --- \n

Here is any available background question + answer pairs:

\n --- \n {q_a_pairs} \n --- \n

Here is additional context relevant to the question: 

\n --- \n {context} \n --- \n

Use the above context and any background question + answer pairs to answer the question: \n {question}
"""

decomposition_prompt = ChatPromptTemplate.from_template(template)

In [67]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

def format_qa_pair(question, answer):
    """Format Q and A pair"""
    
    formatted_string = ""
    formatted_string += f"Question: {question}\nAnswer: {answer}\n\n"
    return formatted_string.strip()

# llm
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

q_a_pairs = ""
for q in questions:
    
    rag_chain = (
    {"context": itemgetter("question") | retriever, 
     "question": itemgetter("question"),
     "q_a_pairs": itemgetter("q_a_pairs")} 
    | decomposition_prompt
    | llm
    | StrOutputParser())

    answer = rag_chain.invoke({"question":q,"q_a_pairs":q_a_pairs})
    q_a_pair = format_qa_pair(q,answer)
    q_a_pairs = q_a_pairs + "\n---\n"+  q_a_pair

In [68]:
answer

'Task decomposition techniques that aid memory enhancement involve breaking down complex tasks into smaller, more manageable subgoals. By focusing on specific components, individuals can improve memory retention and recall. Some examples of task decomposition techniques include:\n\n1. Decomposing a problem into multiple thought steps and generating multiple thoughts per step, creating a tree structure.\n2. Using LLM with simple prompting like "Steps for XYZ" or "What are the subgoals for achieving XYZ?"\n3. Employing task-specific instructions, such as "Write a story outline" for writing a novel.\n4. Involving human inputs in the task decomposition process.\n\nThese techniques help individuals encode and store information more effectively in both short-term and long-term memory, ultimately enhancing memory retention and recall abilities.'

#### Option 2: answer each of the subquestions individually (not using the response to the previous subquestion as context).
* Useful when the subquestions are not dependent from each other.

In [76]:
# Answer each sub-question individually

from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# RAG prompt
prompt_rag_template = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.

Question: {question} 

Context: {context} 

Answer:"""

prompt_rag = ChatPromptTemplate.from_template(prompt_rag_template)


def retrieve_and_rag(question, prompt_rag, sub_question_generator_chain):
    """RAG on each sub-question"""

    # Use our decomposition /
    sub_questions = sub_question_generator_chain.invoke({"question": question})

    # Initialize a list to hold RAG chain results
    rag_results = []

    for sub_question in sub_questions:

        # Retrieve documents for each sub-question
        retrieved_docs = retriever.get_relevant_documents(sub_question)

        # Use retrieved documents and sub-question in RAG chain
        answer = (prompt_rag | llm | StrOutputParser()).invoke(
            {"context": retrieved_docs, "question": sub_question}
        )
        rag_results.append(answer)

    return rag_results, sub_questions


# Wrap the retrieval and RAG process in a RunnableLambda for integration into a chain
answers, questions = retrieve_and_rag(
    question, prompt_rag, generate_queries_decomposition
)
print(answers)
print('=====')
print(questions)

['The key steps involved in task decomposition include instructing the model to "think step by step" to break down complex tasks into smaller steps. This can be achieved through simple prompting by LLM, task-specific instructions, or human inputs. Task decomposition aims to transform big tasks into multiple manageable tasks for better performance.', "Task decomposition helps in improving memory retention by breaking down large tasks into smaller, manageable subgoals. This process enables efficient handling of complex tasks and enhances the agent's ability to retain and recall information over extended periods. Memory retention is further improved through the agent's capability to reflect on past actions, learn from mistakes, and refine strategies for future steps.", 'Memory techniques such as short-term and long-term memory utilization can enhance task decomposition by providing the agent with the ability to retain and recall information over time. These memory capabilities can help in

In [77]:
def format_qa_pairs(questions, answers):
    """Format Q and A pairs"""
    
    formatted_string = ""
    for i, (question, answer) in enumerate(zip(questions, answers), start=1):
        formatted_string += f"Question {i}: {question}\nAnswer {i}: {answer}\n\n"
    return formatted_string.strip()

context = format_qa_pairs(questions, answers)

# Prompt
template = """Here is a set of Q+A pairs:

{context}

Use these to synthesize an answer to the question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"context":context,"question":question})

'Task decomposition involves breaking down complex tasks into smaller, manageable subgoals to improve performance. By instructing the model to think step by step, it can effectively handle large tasks and enhance memory retention. Memory techniques such as short-term and long-term memory utilization can be used in conjunction with task decomposition to further enhance memory retention and improve the quality of final results. By combining task decomposition with memory techniques, agents can efficiently handle complex tasks, retain and recall information over time, learn from past actions, and refine strategies for better outcomes.'

## Query translation technique #4: Step-back questions (also called step-back prompting)
* Step-back questions (step-back prompting): go one step back and make the questions that precede or originate the user's question. Example:
    * User question: where were Mozart born?
    * Step-back question: who was Mozart? 

In [78]:
# Few Shot Examples
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
examples = [
    {
        "input": "Could the members of The Police perform lawful arrests?",
        "output": "what can the members of The Police do?",
    },
    {
        "input": "Jan Sindel’s was born in what country?",
        "output": "what is Jan Sindel’s personal history?",
    },
]
# We now transform these to example messages
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:""",
        ),
        # Few shot examples
        few_shot_prompt,
        # New question
        ("user", "{question}"),
    ]
)


In [81]:
generate_queries_step_back = prompt | llm | StrOutputParser()
question = "What is task decomposition for LLM agents?"
generate_queries_step_back.invoke({"question": question})

'What is the process of breaking down tasks for LLM agents?'

In [82]:
# Response prompt 
response_prompt_template = """You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.

# {normal_context}
# {step_back_context}

# Original Question: {question}
# Answer:"""
response_prompt = ChatPromptTemplate.from_template(response_prompt_template)

chain = (
    {
        # Retrieve context using the normal question
        "normal_context": RunnableLambda(lambda x: x["question"]) | retriever,
        # Retrieve context using the step-back question
        "step_back_context": generate_queries_step_back | retriever,
        # Pass on the question
        "question": lambda x: x["question"],
    }
    | response_prompt
    | llm
    | StrOutputParser()
)

chain.invoke({"question": question})

'Task decomposition for LLM agents involves breaking down complex tasks into smaller and more manageable subtasks. This process allows the LLM (large language model) to better understand the overall task and approach it in a step-by-step manner. By decomposing tasks, the LLM can effectively plan and execute each subtask, leading to more efficient problem-solving and decision-making.\n\nOne common technique used for task decomposition is the Chain of Thought (CoT), where the model is prompted to "think step by step" to decompose hard tasks into simpler steps. This approach helps the LLM transform large and complex tasks into a series of smaller tasks, making it easier to tackle each component individually.\n\nAdditionally, the Tree of Thoughts extends the concept of CoT by exploring multiple reasoning possibilities at each step. It generates multiple thoughts per step, creating a tree structure of task decomposition. This method allows for a more comprehensive analysis of the problem an

## Query Translation Technique #5: The HyDE Technique
* Given the user's question, create a fake document that would answer it properly.
* Then go to the vector database and find documents similar to the fake document. 

In [83]:
# HyDE document generation
template = """Please write a scientific paper passage to answer the question
Question: {question}
Passage:"""
prompt_hyde = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_docs_for_retrieval = (
    prompt_hyde | ChatOpenAI(temperature=0) | StrOutputParser() 
)

# Run
question = "What is task decomposition for LLM agents?"
generate_docs_for_retrieval.invoke({"question":question})

'Task decomposition is a fundamental concept in the field of reinforcement learning and artificial intelligence, particularly for Large Language Models (LLMs) agents. Task decomposition refers to the process of breaking down a complex task into smaller, more manageable sub-tasks or components. This allows the agent to effectively tackle the overall task by focusing on individual components sequentially or in parallel.\n\nIn the context of LLM agents, task decomposition is crucial for improving the efficiency and effectiveness of the learning process. By breaking down a complex task into smaller sub-tasks, LLM agents can better understand the underlying structure and dependencies within the task. This enables the agent to learn more efficiently and make better decisions when faced with complex and ambiguous situations.\n\nFurthermore, task decomposition can also help LLM agents to generalize their learning across different tasks and domains. By decomposing tasks into smaller components,

In [84]:
# Retrieve
retrieval_chain = generate_docs_for_retrieval | retriever 
retireved_docs = retrieval_chain.invoke({"question":question})
retireved_docs

[Document(metadata={'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:', 'language': 'en', 'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log"}, page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performa

In [85]:
# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"context":retireved_docs,"question":question})

'Task decomposition for LLM agents involves breaking down complex tasks into smaller and simpler steps. This process allows the agent to better understand the task at hand and plan ahead effectively. Task decomposition can be achieved through techniques like Chain of Thought (CoT) and Tree of Thoughts, which help the LLM agent explore multiple reasoning possibilities at each step and generate multiple thoughts per step, creating a structured approach to problem-solving. Task decomposition can be facilitated by simple prompting, task-specific instructions, or human inputs.'

# Routing
Purpose: To direct the question to the appropriate data source (such as a vector database, relational database, or graph database) or the appropriate prompt or any other routing option.

Types of Routing:
* Logical Routing: Uses the LLM's knowledge of data sources to decide the best destination for a query.
* Semantic Routing: Involves embedding the question and prompts, calculating similarity, and selecting the prompt with the highest similarity for routing. For example: based on the question, use the Math prompt or the Physics prompt.

#### Logical Routing

In [86]:
from typing import Literal

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

# Data model
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasource: Literal["python_docs", "js_docs", "golang_docs"] = Field(
        ...,
        description="Given a user question choose which datasource would be most relevant for answering their question",
    )

# LLM with function call 
structured_llm = llm.with_structured_output(RouteQuery)

# Prompt 
system = """You are an expert at routing a user question to the appropriate data source.

Based on the programming language the question is referring to, route it to the relevant data source."""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

# Define router 
router = prompt | structured_llm

c:\Users\gaarango\Projectos\Indra\CAPACITACIONES\rag-applications-intro\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [87]:
question = """Why doesn't the following code work:

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(["human", "speak in {language}"])
prompt.invoke("french")
"""

result = router.invoke({"question": question})

In [88]:
result

RouteQuery(datasource='python_docs')

In [89]:
result.datasource

'python_docs'

In [90]:
def choose_route(result):
    if "python_docs" in result.datasource.lower():
        ### Logic here 
        return "chain for python_docs"
    elif "js_docs" in result.datasource.lower():
        ### Logic here 
        return "chain for js_docs"
    else:
        ### Logic here 
        return "golang_docs"

from langchain_core.runnables import RunnableLambda

full_chain = router | RunnableLambda(choose_route)

In [91]:
full_chain.invoke({"question": question})

'chain for python_docs'

#### Semantic Routing

In [92]:
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# Two prompts
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{query}"""

math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{query}"""

# Embed prompts
embeddings = OpenAIEmbeddings()
prompt_templates = [physics_template, math_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)

# Route question to prompt 
def prompt_router(input):
    # Embed question
    query_embedding = embeddings.embed_query(input["query"])
    # Compute similarity
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]
    # Chosen prompt 
    print("Using MATH" if most_similar == math_template else "Using PHYSICS")
    return PromptTemplate.from_template(most_similar)


chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | ChatOpenAI()
    | StrOutputParser()
)

print(chain.invoke("What's a black hole"))

Using PHYSICS
A black hole is a region in space where the gravitational pull is so strong that nothing, not even light, can escape from it. It is formed when a massive star collapses under its own gravity. The boundary around a black hole is called the event horizon, beyond which nothing can escape. Black holes can vary in size and mass, with supermassive black holes being found at the center of galaxies.


# Query structuring

Transform the query from natural language to some query syntax.

In the example below, we transform the query from natural language to the query syntax according to the schema we have defined.

In [ ]:
from langchain_community.document_loaders import YoutubeLoader

docs = YoutubeLoader.from_youtube_url(
    "https://www.youtube.com/watch?v=pbAd8O1Lvm4", add_video_info=True
).load()

docs[0].metadata

In [ ]:
import datetime
from typing import Literal, Optional, Tuple
from langchain_core.pydantic_v1 import BaseModel, Field


class TutorialSearch(BaseModel):
    """Search over a database of tutorial videos about a software library."""

    content_search: str = Field(
        ...,
        description="Similarity search query applied to video transcripts.",
    )
    title_search: str = Field(
        ...,
        description=(
            "Alternate version of the content search query to apply to video titles. "
            "Should be succinct and only include key words that could be in a video "
            "title."
        ),
    )
    min_view_count: Optional[int] = Field(
        None,
        description="Minimum view count filter, inclusive. Only use if explicitly specified.",
    )
    max_view_count: Optional[int] = Field(
        None,
        description="Maximum view count filter, exclusive. Only use if explicitly specified.",
    )
    earliest_publish_date: Optional[datetime.date] = Field(
        None,
        description="Earliest publish date filter, inclusive. Only use if explicitly specified.",
    )
    latest_publish_date: Optional[datetime.date] = Field(
        None,
        description="Latest publish date filter, exclusive. Only use if explicitly specified.",
    )
    min_length_sec: Optional[int] = Field(
        None,
        description="Minimum video length in seconds, inclusive. Only use if explicitly specified.",
    )
    max_length_sec: Optional[int] = Field(
        None,
        description="Maximum video length in seconds, exclusive. Only use if explicitly specified.",
    )

    def pretty_print(self) -> None:
        for field in self.__fields__:
            if getattr(self, field) is not None and getattr(self, field) != getattr(
                self.__fields__[field], "default", None
            ):
                print(f"{field}: {getattr(self, field)}")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

system = """You are an expert at converting user questions into database queries. \
You have access to a database of tutorial videos about a software library for building LLM-powered applications. \
Given a question, return a database query optimized to retrieve the most relevant results.

If there are acronyms or words you are not familiar with, do not try to rephrase them."""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
structured_llm = llm.with_structured_output(TutorialSearch)
query_analyzer = prompt | structured_llm

In [ ]:
#type(out)

In [ ]:
query_analyzer.invoke({"question": "rag from scratch"}).pretty_print()

In [ ]:
query_analyzer.invoke(
    {"question": "videos on chat langchain published in 2023"}
).pretty_print()

In [ ]:
query_analyzer.invoke(
    {
        "question": "how to use multi-modal models in an agent, only videos under 5 minutes"
    }
).pretty_print()